# 1. Extraindo dados do Spotify

## a) Importando as bibliotecas

Obs.: Spotipy é uma biblioteca desenvolvida para ler as APIs do Spotify (https://spotipy.readthedocs.io/en/latest/#)

In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
from json.decoder import JSONDecodeError

## b) informar seus IDs de developer Spotify

In [2]:
cid = 'd779d6da93d24de4838434f87e14bba8' #client id
sid = '9b4b3a170a3c405abe670bb74476931d' #secret id

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=sid)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## c) Buscar as faixas disponíveis no país escolhido por ano de lançamento
Esta busca retornará as músicas, com seu ano de lançamento.

A popularidade de cada faixa é um número de 0 a 100, sendo 100 mais popular e 0 menos popular.

É a popularidade atual da faixa, de acordo com as execuções mais recentes dos usuários no Spotify.

Como há um limite de 2000 faixas a cada chamada, optei por segregar por ano, assim consigo uma amostra muito maior. Como as faixas mais populares aparecem primeiro, e vou me concentrar nas faixas + populares, não é um problema não conseguir os dados totais, que devem ser bem mais numerosos. 

In [4]:
# Criar listas para guardar os dados
artist_name = []
track_name = []
track_id = []
track_preview_url = []
track_popularity = []
track_year = []
album = []
album_art = []
duration_ms = []
release_date = []

# Percorrer os anos de 1960 até 2020
initial_year = 1960
final_year = 2020

market_country = 'DE'

for ano in range(initial_year, (final_year+1)):
    print(ano)
    for i in range(0,2000,50):
        track_results = sp.search(q='year:{}'.format(ano), type='track', limit=50,offset=i, market=market_country)
        for i, t in enumerate(track_results['tracks']['items']):
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            track_preview_url.append(t['preview_url'])
            track_popularity.append(t['popularity'])
            track_year.append(ano)
            album.append(t['album']['name'])
            try:
                album_art.append(t['album']['images'][1]['url']) #há 3 figuras. A primeira c/ maior resolucao, peguei a primeira.
            except Exception:
                album_art.append('not found')
            duration_ms.append(t['duration_ms'])
            release_date.append(t['album']['release_date'])


1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


## d) Guardar os resultados em um DataFrame

In [5]:
pd.options.display.max_colwidth = 100

track_dataframe = pd.DataFrame({'artist_name' : artist_name,
                                'track_name' : track_name,
                                'album' : album,
                                'track_popularity' : track_popularity,
                                'track_id' : track_id,
                                'track_preview_url' : track_preview_url,
                                'album_art' : album_art,
                                'duration_ms' : duration_ms,
                                'track_year' : track_year,
                                'release_date' : release_date})

print(track_dataframe.shape)


(122000, 10)


## e) Eliminar eventuais músicas duplicadas

In [6]:
# eliminar track_id's duplicados
track_dataframe = track_dataframe.drop_duplicates(subset="track_id")
print(track_dataframe.shape)

(121950, 10)


## f) Salvar em um arquivo csv

In [7]:
from datetime import date
hoje = date.today().strftime("%Y-%m-%d")

track_dataframe.to_csv("spotify_tracks_{}_{}.csv".format(hoje, market_country), quotechar='"', escapechar='"')

In [8]:
track_dataframe.sample(5)

,artist_name,track_name,album,track_popularity,track_id,track_preview_url,album_art,duration_ms,track_year,release_date
67777,No Sports,King Kong,King Ska,22,15BssM3kUIOzBGjvYe94Sm,https://p.scdn.co/mp3-preview/9dd0c7a192b30d7d006cc8870958425fd5413744?cid=d779d6da93d24de483843...,https://i.scdn.co/image/ab67616d00001e02e427fae7daf80e36b53ef24d,312133,1993,1993
46714,Bibi Blocksberg,Bibi Blocksberg Lied,Folge 9: verliebt sich,29,4EnEwBUmSJGOCAdHGUiH58,https://p.scdn.co/mp3-preview/ca0e2ff768fbfbc501018091bfb6ac213fbb21e3?cid=d779d6da93d24de483843...,https://i.scdn.co/image/ab67616d00001e026fffb9a579b1ea6499bf7d2f,88873,1983,1983-01-01
71141,Pulp,Mile End,Different Class / Deluxe Edition,49,3DYnqG3jqB1bijSKsJzuzG,None,https://i.scdn.co/image/ab67616d00001e0232b1955183a7fbe0058735fc,270800,1995,1995-10-30
83110,André Rieu,Cavalleria Rusticana: Intermezzo Sinfonico,Dreaming,52,3ILnOG0UZAtf2AtxiJ3vzT,None,https://i.scdn.co/image/ab67616d00001e0243b2cd2f6c8a9fb74aa0a950,207093,2001,2001-01-01
104561,Die drei ???,Der 5. Advent: 13. Dezember (13 von 24) - Teil 02,Der 5. Advent,44,2jkoLaE1KWuvOcMKqfOXfp,https://p.scdn.co/mp3-preview/93e609622cb4723f9fd790dc9692f72001410a4e?cid=d779d6da93d24de483843...,https://i.scdn.co/image/ab67616d00001e025aca6ab743013c0e7539ce06,102160,2012,2012-12-01
